<img src='img/logo.png' />

<img src='img/title.png'>

<img src='img/py3k.png'>

# Table of Contents
* [Context Managers](#Context-Managers)
	* [Exercise](#Exercise)

# Context Managers

In [ ]:
# You may have seen code that opens a file like this
with open('tmp/advPython_test', 'w') as fi:
    fi.write("Hello")

The ```with``` keyword was added in Python 2.5 with PEP343.  These blocks of code are called "context managers".  We set up a context for opening a file and execute a few lines.  As soon as execution leaves the ```with``` block, the file is safely closed.

Context managers are special classes with two magic methods: ```__enter__()``` and ```__exit__()```.  The purpose of these managers is to factor out often used try/finally clauses to make the code more readable.

In [ ]:
# Equivalent to the code above
try:
    fi = open('tmp/advPython_test', 'w')
    fi.write("Hello")
finally:
    fi.close()

Context managers allow blocks of code to have a setup/initization phase, execution, and finally, a cleanup phase.

In [ ]:
class my_context(object):
    def __init__(self):
        print("Initializing context")
        
    def __enter__(self):
        print("Entering context")
        # this becomes bound to the variable in the context manager
        return 42
        
    def __exit__(self, type, value, traceback):
        print("Exiting context")

In [ ]:
with my_context() as t:
    print("The answer is", t)

The ```contextlib``` module in the standard library includes some useful context managers as well as a decorator that can turn a generator into a context manager.  Documentation for the ```contextlib``` module can be found here: https://docs.python.org/3/library/contextlib.html

In [ ]:
import contextlib

@contextlib.contextmanager
def my_context2():
    # Nice idea to wrap in try/except/finally block to have error handling
    try:    
        print("Entering context")
        # This is the where we give control to the block of code
        # in the context manager.
        # The value we yield is bound to the context variable.
        yield 42
    finally:
        print("Exiting context")

In [ ]:
with my_context2() as t:
    print("The answer generated is", t)

We can nest context managers.

```
with manager(), manager1():
    <code block>
```

is equivalent to 
```
with manager():
    with manager1():
       <code block>
```

Some of the interesting context managers in the ```contextlib``` module are:
  * ```closing```: useful for closeable resources.
  * ```ExitStack```: useful for managing a stack of context managers (Python 3 only)

`ExitStack` can help you manage multiple context managers, like opening mulitple files or obtaining multiple locks.  The `ExitStack` context manager also allows you to register custom callback functions.

In [ ]:
@contextlib.contextmanager
def manager(x):
    print("Entering Manager", x)
    yield
    print("Exiting Manager", x)

def callback(x):
    print("Executing callback", x)

with contextlib.ExitStack() as stack:
    stack.enter_context(manager(0))
    stack.callback(callback, 0)
    stack.enter_context(manager(1))
    stack.enter_context(manager(2))
    print("Should I do some work here?")

## Exercise

In this exercise we will use Python's `timeit` module to write a context manager that will time the execution of a block of code.

With Python's timeit module, we can time small snippets of code.

In [ ]:
from timeit import timeit
timeit('min(range(1000))', number=1000)

`timeit.timeit` can also accept a callable object that accepts no arguments.  The one important detail is the function that takes no arguments.  Typically the functions that we want to time require arguments.  Is there a way we can "freeze" the arguments of a function?  Turns out that, there is a very nice solution.  We can use either `lambda` functions or `partial` functions (from `functools`).

The idea is we want to create a new function that is equivalent to another function called with a specific set of arguements.
```python
def g(a, b, c):
    return a**2 + b**2 == c**2
    
def h():
    return g(3, 4, 5)
```

We can say that `h()` is equivalent to `g(a=3, b=4, c=5)`.
Below are more general ways of accomplishing this argument "freeze"

In [ ]:
func = min
args = (range(1000),)
kwargs = {}

In [ ]:
h = lambda: func(*args, **kwargs)
timeit(h, number=1000)

In [ ]:
from functools import partial
h = partial(func, *args, **kwargs)
timeit(h, number=1000)

We want to write a context manager that will allow us to time aribitrary functions with arbitrary arguments and keyword arguments.  The goal is to make such timings natural and easy-to-do.

```python
with Timer(number=1000) as t:
    t.time(func, *args, **kwargs)
t.results
```

In [ ]:
from functools import partial
from timeit import timeit

class Timer(object):
    def __init__(self, loops=5, gc=False):
        self.loops = 5
        self.gc = gc
        self.results = defaultdict(list)
        
    def __enter__(self):
        return self
    
    def __exit__(self, type_, value, traceback):
        pass
    
    def time(self, func, *args, **kwargs):
        if self.gc:
            setup = "gc.enable()"
        else:
            setup = "gc.disable()"
        
        
        funcname = func.__name__
        if funcname == "<lambda>":
            funcname = func.__repr__()
            
        pfunc = partial(func, *args, **kwargs)
        elapsed = timeit(pfunc, setup, number=self.loops)
        runtime = min(elapsed)/self.loops
        self.results[funcname].append(runtime)
        return runtime

<img src='img/copyright.png'>